# 1. Load config and variables

In [1]:
%config IPCompleter.greedy=True
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import os

import spacy, pickle

import torch

from torchtext import data
from torchtext import datasets

import random
import inspect

# Custom impport
from common.common_classes import TensorField

In [3]:
torch.cuda.is_available()

True

In [4]:
path = "./"
save_data_path = path + 'save_data/'
large_save_data_path = '/notebooks/large-storage/'
saved_models_path = '/notebooks/large-storage/saved-models/'
print(os.listdir(path))

['test.tsv', 'prepare-word-embedding-nlp.ipynb', 'tokenization.ipynb', 'test-batching-padding.ipynb', 'train.tsv', 'test-batching-padding-ok.ipynb', 'sampleSubmission.csv', 'save_data', '.ipynb_checkpoints', '__init__.py', 'README.md', '.gitignore', '.git', 'common', 'simple-GRU-implement.ipynb']


In [5]:
loaded_data = pickle.load(open(save_data_path + 'pre-processed-data.pkl', 'rb'))
loaded_kaggle_test = pickle.load(open(save_data_path + 'pre-processed-kaggle-test.pkl', 'rb'))
loaded_vocab = pickle.load(open(save_data_path + 'genereated-vocab.pkl', 'rb'))

In [6]:
loaded_data.head()

,PhraseId,SentenceId,Phrase,Sentiment,Phrase_length,Tokenized_phrase,Indexed_phrase
0,1,1,A series of escapades demonstrating the adage ...,1,188,"[xxbos, a, series, of, escapades, demonstratin...","[2, 10, 341, 11, 14246, 6044, 8, 6604, 19, 64,..."
1,2,1,A series of escapades demonstrating the adage ...,2,77,"[xxbos, a, series, of, escapades, demonstratin...","[2, 10, 341, 11, 14246, 6044, 8, 6604, 19, 64,..."
2,3,1,A series,2,8,"[xxbos, a, series, xxeos]","[2, 10, 341, 3]"
3,4,1,A,2,1,"[xxbos, a, xxeos]","[2, 10, 3]"
4,5,1,series,2,6,"[xxbos, series, xxeos]","[2, 341, 3]"


In [7]:
loaded_kaggle_test.head()

,PhraseId,SentenceId,Phrase,Phrase_length,Tokenized_phrase,Indexed_phrase
0,156061,8545,An intermittently pleasing but mostly routine ...,188,"[xxbos, xxmaj, an, intermittently, pleasing, b...","[2, 7, 26, 2606, 1723, 30, 632, 1041, 409, 15, 3]"
1,156062,8545,An intermittently pleasing but mostly routine ...,77,"[xxbos, xxmaj, an, intermittently, pleasing, b...","[2, 7, 26, 2606, 1723, 30, 632, 1041, 409, 3]"
2,156063,8545,An,8,"[xxbos, xxmaj, an, xxeos]","[2, 7, 26, 3]"
3,156064,8545,intermittently pleasing but mostly routine effort,1,"[xxbos, intermittently, pleasing, but, mostly,...","[2, 2606, 1723, 30, 632, 1041, 409, 3]"
4,156065,8545,intermittently pleasing but mostly routine,6,"[xxbos, intermittently, pleasing, but, mostly,...","[2, 2606, 1723, 30, 632, 1041, 3]"


In [8]:
UNK, PAD, BOS, EOS, TK_REP, TK_WREP, TK_UP, TK_MAJ = "xxunk xxpad xxbos xxeos xxrep xxwrep xxup xxmaj".split()

default_spec_tok = [UNK, PAD, BOS, EOS, TK_REP, TK_WREP, TK_UP, TK_MAJ]

MAX_LABEL = 5

# 2. Encoding and prepraing batches

In [9]:
nlp = spacy.load(large_save_data_path + 'process-spacy-model')

In [10]:
nlp.vocab[BOS].vector

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [11]:
type(nlp.vocab.get_vector('test'))

numpy.ndarray

In [12]:
nlp.vocab.vectors.data.shape

(890280, 308)

In [13]:
# For Kaggle Train dataset
PHRASE_ID = data.Field(use_vocab = False)
TEXT = TensorField(include_lengths = True, use_vocab = False, sequential = False, pad_token = nlp.vocab[PAD].vector, dtype=torch.float)
LABEL = data.LabelField(use_vocab = False, dtype=torch.long)

# For Kaggle Test dataset
ID_TEST = data.Field(use_vocab = False)
PHRASE_ID_TEST = data.Field(use_vocab = False)
TEXT_TEST = TensorField(include_lengths = True, use_vocab = False, sequential = False, pad_token = nlp.vocab[PAD].vector, dtype=torch.float)

In [14]:
# For Kaggle Train dataset
fields = [('id', PHRASE_ID), ('text', TEXT), ('label', LABEL)]

# For Kaggle Test dataset
fields_test = [('id', ID_TEST), ('phrase_id', PHRASE_ID_TEST), ('text', TEXT_TEST)]

In [15]:
len(loaded_data['Phrase'])

156060

In [16]:
len(loaded_kaggle_test['Phrase'])

66292

In [17]:
# For Kaggle Train dataset
examples = []
length = len(loaded_data['Phrase'])
for i in range(length):
    embedded = []
    for j in range(len(loaded_data['Tokenized_phrase'][i])):
        if nlp.vocab.has_vector(loaded_data['Tokenized_phrase'][i][j]):
            embedded.append(nlp.vocab.get_vector(loaded_data['Tokenized_phrase'][i][j]))
        else:
            embedded.append(nlp.vocab.get_vector(UNK))
    
    examples.append(data.Example.fromlist([ [loaded_data['PhraseId'][i]], embedded, loaded_data['Sentiment'][i]], fields))
    
examples[:10]

In [18]:
# For Kaggle Test dataset
examples_test = []
length = len(loaded_kaggle_test['Phrase'])
for i in range(length):
    embedded = []
    for j in range(len(loaded_kaggle_test['Tokenized_phrase'][i])):
        if nlp.vocab.has_vector(loaded_kaggle_test['Tokenized_phrase'][i][j]):
            embedded.append(nlp.vocab.get_vector(loaded_kaggle_test['Tokenized_phrase'][i][j]))
        else:
            embedded.append(nlp.vocab.get_vector(UNK))
    
    examples_test.append(data.Example.fromlist([ [i], [loaded_kaggle_test['PhraseId'][i]], embedded ], fields_test))
    
examples_test[:10]

In [19]:
examples[3].text

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 

In [20]:
examples_test[3].text

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 

In [21]:
nlp.vocab.get_vector('a')

array([ 4.3798e-02,  2.4779e-02, -2.0937e-01,  4.9745e-01,  3.6019e-01,
       -3.7503e-01, -5.2078e-02, -6.0555e-01,  3.6744e-02,  2.2085e+00,
       -2.3389e-01, -6.8360e-02, -2.2355e-01, -5.3989e-02, -1.5198e-01,
       -1.7319e-01,  5.3355e-02,  1.6485e+00, -4.7991e-02, -8.5311e-02,
       -1.5712e-01, -6.4425e-01, -3.9819e-01,  2.7800e-01,  1.5364e-01,
        3.1678e-02,  5.5414e-02,  1.5939e-02,  3.1851e-01, -5.8979e-02,
        3.8584e-02,  1.0770e-01,  1.0410e-01, -7.7346e-02,  3.7396e-01,
       -2.1482e-01,  3.8320e-01, -2.7737e-01, -1.8352e-01, -8.3838e-01,
        3.4124e-01,  5.8164e-01,  1.8543e-01, -3.1028e-01,  1.7666e-01,
       -6.9421e-02, -3.4422e-01, -1.3665e-01, -1.0823e-01,  2.3637e-01,
       -3.2923e-01,  6.1348e-01,  1.9720e-01,  8.7123e-02,  1.0785e-01,
        3.0730e-01,  1.3757e-01,  3.0809e-01,  2.4331e-01, -2.9422e-01,
       -9.8214e-03,  5.5675e-01, -4.8880e-02,  9.9468e-02,  3.0543e-01,
       -3.7597e-01, -1.9525e-01,  4.6246e-02, -3.6675e-02,  3.40

In [22]:
examples[3].label

2

In [23]:
len(examples)

156060

In [24]:
len(examples_test)

66292

In [25]:
data_set = data.Dataset(examples, fields)
data_set_test = data.Dataset(examples_test, fields_test)

In [26]:
data_set.sort_key = lambda x: len(x.text)
data_set_test.sort_key = lambda x: len(x.text)

In [27]:
SEED = 9131

In [28]:
train_data, valid_data, test_data = data_set.split([0.8, 0.1, 0.1], random_state = random.seed(SEED))

In [29]:
train_data.sort_key

<function __main__.<lambda>(x)>

In [30]:
len(data_set)*0.1

15606.0

In [31]:
len(train_data)

124848

In [32]:
len(valid_data)

15606

In [33]:
len(test_data)

15606

In [34]:
len(data_set_test)

66292

In [35]:
inspect.getsource(train_data.sort_key)

'data_set.sort_key = lambda x: len(x.text)\n'

In [36]:
inspect.getsource(data_set_test.sort_key)

'data_set_test.sort_key = lambda x: len(x.text)\n'

## Define Hyperparameter

In [37]:
BATCH_SIZE = 64

In [38]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

kaggle_test_iterator = data.BucketIterator(
    data_set_test, 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

In [39]:
raw_train_iterator = iter(train_iterator)
batch = next(raw_train_iterator)

raw_kaggle_test_iterator = iter(kaggle_test_iterator)
batch_test = next(raw_kaggle_test_iterator)

In [40]:
a, b = batch.text

c, d = batch_test.text

In [41]:
b

tensor([9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9.,
        9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9.,
        9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9.,
        9., 9., 9., 9., 9., 9., 9., 9., 9., 9.], device='cuda:0')

In [42]:
a.shape

torch.Size([64, 9, 308])

In [84]:
batch.label

tensor([2, 2, 0, 2, 2, 1, 3, 3, 1, 3, 3, 2, 2, 2, 2, 2, 0, 2, 1, 4, 3, 2, 3, 2,
        2, 2, 1, 3, 1, 2, 2, 1, 3, 1, 2, 2, 1, 4, 2, 3, 2, 3, 2, 2, 1, 4, 2, 2,
        2, 3, 2, 3, 1, 2, 2, 3, 4, 2, 2, 4, 2, 2, 1, 4], device='cuda:0')

In [44]:
batch.id

tensor([[133918,  57783,  94049,  86671,  84086,  74128, 112683, 126943,  12599,
         138419,  47937,  62635, 104584,  86587, 113162,  81036,  62472, 147974,
          32209, 133654,  95241, 137072,  66415,  75555,  53039, 130485,  79837,
         152257,  39951, 136680, 121126, 102615, 101662, 119728, 108929, 145776,
          36689,  66928, 109732, 109843, 130506,  33026, 123268, 155495, 114095,
          27453, 122599,  82520,  21535, 104973,  91096,  97182,  84542, 119081,
         104435,  65401,  30863, 141808,  39997, 145932,  46149, 125167, 132902,
         117864]], device='cuda:0')

In [45]:
loaded_data['Sentiment'][117864-1]

4

In [46]:
d

tensor([10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10.], device='cuda:0')

In [47]:
c.shape

torch.Size([64, 10, 308])

In [80]:
batch_test.id.shape

torch.Size([1, 64])

In [81]:
batch_test.phrase_id.shape

torch.Size([1, 64])

In [51]:
loaded_kaggle_test.iloc[37401]

PhraseId                                                       193462
SentenceId                                                      10289
Phrase                         you 're not into the Pokemon franchise
Phrase_length                                                      72
Tokenized_phrase    [xxbos, you, 're, not, into, the, xxmaj, pokem...
Indexed_phrase               [2, 33, 157, 41, 61, 8, 7, 2893, 978, 3]
Name: 37401, dtype: object

In [78]:
c[63][2]

tensor([-1.2598e-01,  2.3681e-01, -3.0048e-01, -3.4822e-02,  1.4940e-01,
        -3.6147e-02, -2.9160e-02,  4.2446e-02, -1.4922e-02,  2.4803e+00,
         1.3138e-01,  1.8270e-01,  1.9732e-01, -1.1765e-01, -1.9910e-01,
        -9.9976e-02, -1.3003e-02,  8.6874e-01, -2.5387e-01, -4.7935e-02,
         6.2166e-02, -6.8214e-02,  9.0818e-02,  1.8553e-01, -4.6234e-01,
        -7.4295e-02, -1.1136e-01, -1.7341e-01,  6.2637e-01, -5.4815e-01,
        -2.3186e-01,  2.1671e-01,  2.3657e-01,  3.4814e-03,  2.1973e-01,
         8.1694e-02,  7.2784e-02,  3.2418e-01,  1.8485e-01, -7.8116e-02,
        -1.0398e-01,  2.5184e-01, -3.2611e-01,  2.1050e-01,  1.7217e-01,
         1.4633e-01, -2.4610e-01, -1.2313e-01,  1.8549e-02, -1.7679e-01,
         5.4608e-02,  1.9721e-02, -1.4351e-01, -9.8868e-02,  1.9156e-01,
        -1.8175e-01, -9.6741e-03, -2.8860e-01,  3.2727e-01,  8.7877e-02,
         1.5791e-01, -5.9451e-01, -4.0692e-01,  1.0203e-01,  2.0662e-01,
        -3.0393e-01, -1.7494e-01,  1.6815e-01,  1.3

In [60]:
nlp.vocab.get_vector("'re")

array([-1.2598e-01,  2.3681e-01, -3.0048e-01, -3.4822e-02,  1.4940e-01,
       -3.6147e-02, -2.9160e-02,  4.2446e-02, -1.4922e-02,  2.4803e+00,
        1.3138e-01,  1.8270e-01,  1.9732e-01, -1.1765e-01, -1.9910e-01,
       -9.9976e-02, -1.3003e-02,  8.6874e-01, -2.5387e-01, -4.7935e-02,
        6.2166e-02, -6.8214e-02,  9.0818e-02,  1.8553e-01, -4.6234e-01,
       -7.4295e-02, -1.1136e-01, -1.7341e-01,  6.2637e-01, -5.4815e-01,
       -2.3186e-01,  2.1671e-01,  2.3657e-01,  3.4814e-03,  2.1973e-01,
        8.1694e-02,  7.2784e-02,  3.2418e-01,  1.8485e-01, -7.8116e-02,
       -1.0398e-01,  2.5184e-01, -3.2611e-01,  2.1050e-01,  1.7217e-01,
        1.4633e-01, -2.4610e-01, -1.2313e-01,  1.8549e-02, -1.7679e-01,
        5.4608e-02,  1.9721e-02, -1.4351e-01, -9.8868e-02,  1.9156e-01,
       -1.8175e-01, -9.6741e-03, -2.8860e-01,  3.2727e-01,  8.7877e-02,
        1.5791e-01, -5.9451e-01, -4.0692e-01,  1.0203e-01,  2.0662e-01,
       -3.0393e-01, -1.7494e-01,  1.6815e-01,  1.3331e-01,  2.05

In [61]:
torch.cuda.is_available()

True

## Define model

In [62]:
import torch.nn as nn

class GRU(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout):
        
        super().__init__()
        
        self.rnn = nn.GRU( embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional,
                           batch_first=True,
                           dropout=dropout)
        
        self.bidirectional = bidirectional
        
        if bidirectional:
            self.fc = nn.Linear(hidden_dim * 2, output_dim)
        else:
            self.fc = nn.Linear(hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [batch size, sent len, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(text, text_lengths, batch_first=True)
        
        packed_output, hidden = self.rnn(packed_embedded)
        
        #unpack sequence
        #output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        if self.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
        
        #if self.bidirectional:
        #    hidden = [batch size, hid dim * num directions]
        #else:
        #    hidden = [batch size, hid dim]
        return self.fc(hidden)

## Define Hyperparameter and init model

In [63]:
EMBEDDING_DIM = 308
HIDDEN_DIM = 256
OUTPUT_DIM = 5
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5

LEARNING_RATE = 0.0004

N_EPOCHS = 60

MODEL_SAVE_FILE = 'simple-GRU_origin.pt'
model = GRU(EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT).to(device)

### Print out the number of parameters in our model.

In [64]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,054,661 trainable parameters


## Train the Model

In [66]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [67]:
criterion = torch.nn.CrossEntropyLoss().to(device)

### Train function

In [46]:
def train(model, iterator, optimizer, criterion, set_length):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths)
        loss = criterion(predictions, batch.label)
        
        epoch_acc += (predictions.argmax(1) == batch.label).sum().item()
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / set_length, epoch_acc / set_length

### Evaluate function

In [68]:
def evaluate(model, iterator, criterion, set_length):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths)
            
            loss = criterion(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += (predictions.argmax(1) == batch.label).sum().item()
        
    return epoch_loss / set_length, epoch_acc / set_length

### Define epoch time function

In [48]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
best_valid_loss = float('inf')
best_epoch = 0
for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, len(train_data))
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, len(valid_data))
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        best_epoch = epoch
        torch.save(model.state_dict(), saved_models_path + MODEL_SAVE_FILE)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.5f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.5f} |  Val. Acc: {valid_acc*100:.2f}%')

print(f'Best epoch: {best_epoch+1:02}')

Epoch: 01 | Epoch Time: 1m 10s
	Train Loss: 0.01436 | Train Acc: 61.77%
	 Val. Loss: 0.01366 |  Val. Acc: 63.67%
Epoch: 02 | Epoch Time: 1m 11s
	Train Loss: 0.01309 | Train Acc: 64.95%
	 Val. Loss: 0.01271 |  Val. Acc: 66.22%
Epoch: 03 | Epoch Time: 1m 10s
	Train Loss: 0.01238 | Train Acc: 66.95%
	 Val. Loss: 0.01230 |  Val. Acc: 67.12%
Epoch: 04 | Epoch Time: 1m 10s
	Train Loss: 0.01178 | Train Acc: 68.65%
	 Val. Loss: 0.01212 |  Val. Acc: 67.92%
Epoch: 05 | Epoch Time: 1m 10s
	Train Loss: 0.01124 | Train Acc: 70.03%
	 Val. Loss: 0.01187 |  Val. Acc: 68.60%


In [50]:
len(test_data)

15606

In [51]:
print(f'Best epoch: {best_epoch+1:02}')

7

## Test the Model

In [72]:
model.load_state_dict(torch.load(saved_models_path + MODEL_SAVE_FILE))

test_loss, test_acc = evaluate(model, test_iterator, criterion, len(test_data))

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.012 | Test Acc: 68.49%


## Test on Kaggle Dataset and create submittion file

In [107]:
# Change the model target file
MODEL_SAVE_FILE_TARGET = 'simple-GRU-valid-69.59.pt'

def predict_kaggle_test(model, iterator):
    
    result = []
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths)
            
            predictions = predictions.argmax(1)
            
            for i in range(len(batch)):
                result.append([batch.id[0][i].item(), [batch.phrase_id[0][i].item(), predictions[i].item()]] )
    
    result.sort(key = lambda val: val[0])
        
    return result

In [108]:
model.load_state_dict(torch.load(saved_models_path + MODEL_SAVE_FILE_TARGET))

kaggle_result_list = predict_kaggle_test(model, kaggle_test_iterator)

kaggle_result_list[:10]

[[0, [156061, 2]],
 [1, [156062, 2]],
 [2, [156063, 2]],
 [3, [156064, 2]],
 [4, [156065, 2]],
 [5, [156066, 2]],
 [6, [156067, 3]],
 [7, [156068, 2]],
 [8, [156069, 3]],
 [9, [156070, 2]]]

In [112]:
kaggle_result_list[-10:]

[[66282, [222343, 2]],
 [66283, [222344, 2]],
 [66284, [222345, 2]],
 [66285, [222346, 2]],
 [66286, [222347, 2]],
 [66287, [222348, 0]],
 [66288, [222349, 0]],
 [66289, [222350, 1]],
 [66290, [222351, 1]],
 [66291, [222352, 1]]]

In [113]:
len(kaggle_result_list)

66292

In [110]:
loaded_kaggle_test.tail(10)

,PhraseId,SentenceId,Phrase,Phrase_length,Tokenized_phrase,Indexed_phrase
66282,222343,11854,should have called it Gutterball,14,"[xxbos, should, have, called, it, xxmaj, gutte...","[2, 156, 49, 775, 20, 7, 0, 3]"
66283,222344,11854,have called it Gutterball,65,"[xxbos, have, called, it, xxmaj, gutterball, x...","[2, 49, 775, 20, 7, 0, 3]"
66284,222345,11854,called it Gutterball,63,"[xxbos, called, it, xxmaj, gutterball, xxeos]","[2, 775, 20, 7, 0, 3]"
66285,222346,11854,it Gutterball,38,"[xxbos, it, xxmaj, gutterball, xxeos]","[2, 20, 7, 0, 3]"
66286,222347,11854,Gutterball,34,"[xxbos, xxmaj, gutterball, xxeos]","[2, 7, 0, 3]"
66287,222348,11855,"A long-winded , predictable scenario .",32,"[xxbos, a, long, -, winded, ,, predictable, sc...","[2, 10, 139, 13, 5931, 9, 390, 2021, 15, 3]"
66288,222349,11855,"A long-winded , predictable scenario",29,"[xxbos, a, long, -, winded, ,, predictable, sc...","[2, 10, 139, 13, 5931, 9, 390, 2021, 3]"
66289,222350,11855,"A long-winded ,",20,"[xxbos, a, long, -, winded, ,, xxeos]","[2, 10, 139, 13, 5931, 9, 3]"
66290,222351,11855,A long-winded,24,"[xxbos, a, long, -, winded, xxeos]","[2, 10, 139, 13, 5931, 3]"
66291,222352,11855,predictable scenario,21,"[xxbos, predictable, scenario, xxeos]","[2, 390, 2021, 3]"


### Write into CSV Files

In [116]:
CSV_EXTENSION = '.submit.csv'

import csv

with open(saved_models_path + MODEL_SAVE_FILE_TARGET + CSV_EXTENSION, mode='w') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    for i in range(len(kaggle_result_list)):
        csv_writer.writerow(kaggle_result_list[i][1])